# Applied Machine Learning - Mini Challenge: Cross-Selling of Credit Cards
**Author**: Nils Fahrni

In [2]:
import numpy as np
import sys
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

pio.templates.default = 'ggplot2'
pd.set_option('display.max_columns', None)

sys.path.append('scripts')
from dataloader import DataLoader

## Data Preprocessing

### Data Loading

In [3]:
data_loader = DataLoader(base_path='data', translations_name='translation_mappings.json')
data_loader.list_datasets()

,Dataset,Number of Rows
0,loan,682
1,client,5369
2,district,77
3,trans,1056320
4,account,4500
5,card,892
6,order,6471
7,disp,5369


#### Account

In [4]:
account = data_loader.load_csv('account', parse_dates={'date': '%y%m%d'})
account.sample(5)

Mapped frequency:
{
    "POPLATEK MESICNE": "MONTHLY CHARGES",
    "POPLATEK TYDNE": "WEEKLY CHARGES",
    "POPLATEK PO OBRATU": "TRANSACTION CHARGES"
}


,account_id,district_id,frequency,date
2149,2550,67,MONTHLY CHARGES,1995-11-11
4465,938,70,MONTHLY CHARGES,1997-12-13
4122,2587,1,MONTHLY CHARGES,1997-07-25
788,1091,16,MONTHLY CHARGES,1993-09-09
2824,848,31,MONTHLY CHARGES,1996-06-16


#### Client

In [5]:
client = data_loader.load_csv('client')

client = client.assign(gender=client['birth_number'].apply(lambda x: 'FEMALE' if int(str(x)[2:4]) > 50 else 'MALE'))

client = client.assign(birth_number=client.apply(lambda x: x['birth_number'] - 5000 if x['gender'] == 'FEMALE' else x['birth_number'], axis=1))

client['birth_number'] = pd.to_datetime(client['birth_number'], format='%y%m%d')
client['birth_date'] = client['birth_number'].apply(lambda x: x - pd.DateOffset(years=100) if x.year > 1999 else x)
client.drop('birth_number', axis=1, inplace=True)

client['age'] = (pd.to_datetime('1999-12-31') - client['birth_date']).dt.days // 365

client.sample(5)

,client_id,district_id,gender,birth_date,age
236,252,37,FEMALE,1969-09-01,30
5344,13640,28,FEMALE,1955-06-09,44
3472,3664,69,MALE,1951-03-19,48
2036,2153,36,FEMALE,1949-05-16,50
4484,4744,6,MALE,1929-12-18,70


#### Disposition

Removing disponents as the goal is to only advertise to owners. Disponents may be secondary users that have been authorized to use an account. They may be allowed to execute transactions on that account but they are not the authorized owners.

In [6]:
disp = data_loader.load_csv('disp')

disp = disp[disp['type'] != 'DISPONENT']

disp.drop('type', axis=1, inplace=True)

disp.sample(5)

,disp_id,client_id,account_id
2073,2190,2190,1808
564,591,591,492
2378,2507,2507,2068
3418,3608,3608,2988
1314,1385,1385,1152


#### Permanent Order

In [7]:
order = data_loader.load_csv('order')

order.sample(5)

Mapped k_symbol:
{
    "POJISTNE": "INSURANCE PAYMENT",
    "SIPO": "HOUSEHOLD",
    "LEASING": "LEASING",
    "UVER": "LOAN PAYMENT"
}


,order_id,account_id,bank_to,account_to,amount,k_symbol
1704,31287,1287,CD,8041507,525.0,INSURANCE PAYMENT
4054,33880,3025,EF,14488661,3354.0,HOUSEHOLD
3456,33218,2581,OP,81132262,4769.0,HOUSEHOLD
2053,31674,1545,KL,98522644,2625.0,HOUSEHOLD
1645,31220,1244,IJ,81744551,2122.0,NaN


**Are there Null Values?**

In [8]:
display(order.isnull().sum())

display(order[order['k_symbol'].isnull()].sample(5))

order_id         0
account_id       0
bank_to          0
account_to       0
amount           0
k_symbol      1379
dtype: int64

,order_id,account_id,bank_to,account_to,amount,k_symbol
1894,31496,1414,MN,8757979,5397.0,NaN
3526,33291,2642,IJ,37776649,571.0,NaN
569,30021,405,WX,11276412,1497.0,NaN
2598,32263,1957,QR,27709353,892.0,NaN
3418,33180,2550,MN,80567723,403.0,NaN


In [9]:
order['k_symbol'] = order['k_symbol'].fillna('MISSING')

order_pivot = order.pivot_table(index='account_id', columns='k_symbol', values='amount', aggfunc='count', fill_value=0)

order_pivot.sample(5)

k_symbol,HOUSEHOLD,INSURANCE PAYMENT,LEASING,LOAN PAYMENT,MISSING
account_id,,,,,
2872,1,0,0,1,1
1910,1,0,0,0,1
1021,1,0,0,0,0
3,1,1,0,0,1
27,1,0,0,0,0


#### Transaction

TODO: 
- amount to negative or positive based on if withdrawal or deposit
- Research account number 19 (time series account balance, at least per month)
    - this acc is volatile
    - account's balance goes negative sometimes
- what happens to accounts with multiple transactions on a day?
    - how to obtain the actual end of day balance?
        - add up withdrawals with deposits and add to balance of day before 
        - try to vectorize this problem (R antijoin mentioned)

In [10]:
transaction = data_loader.load_csv('trans', parse_dates={'date': '%y%m%d'})

transaction.sample(5)

Mapped type:
{
    "PRIJEM": "CREDIT",
    "VYDAJ": "WITHDRAWAL"
}
Mapped operation:
{
    "VYBER KARTOU": "CREDIT CARD WITHDRAWAL",
    "VKLAD": "CREDIT IN CASH",
    "PREVOD Z UCTU": "COLLECTION FROM ANOTHER BANK",
    "VYBER": "WITHDRAWAL IN CASH",
    "PREVOD NA UCET": "REMITTANCE TO ANOTHER BANK"
}
Mapped k_symbol:
{
    "POJISTNE": "INSURANCE PAYMENT",
    "SLUZBY": "PAYMENT FOR STATEMENT",
    "UROK": "INTEREST CREDITED",
    "SANKC. UROK": "SANCTION INTEREST IF NEGATIVE BALANCE",
    "SIPO": "HOUSEHOLD",
    "DUCHOD": "OLD-AGE PENSION",
    "UVER": "LOAN PAYMENT"
}


,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
54399,834990,2841,1994-04-30,WITHDRAWAL,WITHDRAWAL IN CASH,14.6,25274.0,PAYMENT FOR STATEMENT,NaN,NaN
445214,3632635,3092,1996-12-31,CREDIT,NaN,88.9,22032.2,INTEREST CREDITED,NaN,NaN
857607,597513,2033,1998-05-09,WITHDRAWAL,WITHDRAWAL IN CASH,3500.0,37939.8,NaN,NaN,NaN
537262,674193,2303,1997-04-30,WITHDRAWAL,WITHDRAWAL IN CASH,14.6,39224.9,PAYMENT FOR STATEMENT,NaN,NaN
907733,815215,2776,1998-07-06,WITHDRAWAL,REMITTANCE TO ANOTHER BANK,1647.0,14517.3,HOUSEHOLD,QR,40267509.0


#### Loan

In [11]:
loan = data_loader.load_csv('loan', parse_dates={'date': '%y%m%d'})

loan.sample(5)

Mapped status:
{
    "A": "contract finished, no problems",
    "B": "contract finished, loan not payed",
    "C": "running contract, OK so far",
    "D": "running contract, client in debt"
}


,loan_id,account_id,date,amount,duration,payments,status
344,6028,5072,1997-02-09,317520,60,5292.0,"running contract, OK so far"
611,6450,7099,1998-06-10,234336,48,4882.0,"running contract, OK so far"
392,6887,9263,1997-05-08,39576,12,3298.0,"contract finished, no problems"
579,6136,5572,1998-04-19,140880,60,2348.0,"running contract, OK so far"
466,5719,3664,1997-09-20,271584,48,5658.0,"running contract, OK so far"


**Can an account have multiple loans?**

In [12]:
print(f'Are there accounts with multiple loans: {loan["account_id"].nunique() < loan.shape[0]}')

Are there accounts with multiple loans: False


#### Credit Card

In [13]:
card = data_loader.load_csv('card', parse_dates={'issued': '%y%m%d'})

card.sample(5)

,card_id,disp_id,type,issued
77,113,629,junior,1995-11-16
391,94,513,classic,1997-11-06
88,106,592,classic,1996-01-14
138,24,153,junior,1996-08-05
178,441,2785,classic,1996-11-05


#### District

In [14]:
district = data_loader.load_csv('district')

district = district.rename(columns={
    'A1': 'district_id',
    'A2': 'district_name',
    'A3': 'region',
    'A4': 'population',
    'A5': 'n_municipalities_with_inhabitants_lt_499',
    'A6': 'n_municipalities_with_inhabitants_500_to_1999',
    'A7': 'n_municipalities_with_inhabitants_2000_to_9999',
    'A8': 'n_municipalities_with_inhabitants_gt_10000',
    'A9': 'n_cities',
    'A10': 'ratio_urban_inhabitants',
    'A11': 'average_salary',
    'A12': 'unemployment_rate_95',
    'A13': 'unemployment_rate_96',
    'A14': 'enterpreneurs_per_1000_inhabitants',
    'A15': 'n_commited_crimes_95',
    'A16': 'n_commited_crimes_96'
})

district.sample(5)

,district_id,district_name,region,population,n_municipalities_with_inhabitants_lt_499,n_municipalities_with_inhabitants_500_to_1999,n_municipalities_with_inhabitants_2000_to_9999,n_municipalities_with_inhabitants_gt_10000,n_cities,ratio_urban_inhabitants,average_salary,unemployment_rate_95,unemployment_rate_96,enterpreneurs_per_1000_inhabitants,n_commited_crimes_95,n_commited_crimes_96
52,53,Blansko,south Moravia,107911,88,33,6,2,7,50.9,8240,2.53,3.56,99,1850,1903
36,37,Litomerice,north Bohemia,114006,71,26,6,2,9,62.3,9065,4.46,5.39,123,4147,4166
61,62,Uherske Hradiste,south Moravia,145688,22,41,13,2,5,40.7,8544,1.29,1.86,109,2719,2906
56,57,Hodonin,south Moravia,161954,21,37,20,3,8,48.0,8720,3.73,4.50,116,3729,3651
50,51,Trutnov,east Bohemia,121947,37,28,7,3,11,70.5,8541,2.51,2.97,131,3496,3839


### Data Merging

In [15]:
from utils import add_prefix_except_id

account = add_prefix_except_id(account, 'account_', id_exceptions=['district_id'])
client_df = disp.merge(account, on='account_id', how='left')

client = add_prefix_except_id(client, 'client_', id_exceptions=['district_id'])
client_df = client_df.merge(client, on='client_id', how='left')

order_pivot = add_prefix_except_id(order_pivot, 'ordertype_')
client_df = client_df.merge(order_pivot, on='account_id', how='left')

loan = add_prefix_except_id(loan, 'loan_')
client_df = client_df.merge(loan, on='account_id', how='left')

card = add_prefix_except_id(card, 'card_')
client_df = client_df.merge(card, on='disp_id', how='left')

client_district = add_prefix_except_id(district, 'client_district_')
client_df = client_df.merge(client_district, left_on='client_district_id', right_on='district_id', how='left')

account_district = add_prefix_except_id(district, 'account_district_')
client_df = client_df.merge(account_district, left_on='account_district_id', right_on='district_id', how='left')

client_df.sample(5)

n_merged_base_client = client_df.shape[0]

In [16]:
assert client_df['account_id'].nunique() == client_df.shape[0]

### Data Cleaning

#### Removing Junior Cards

In [17]:
junior_clients = client_df[client_df['card_type'] == 'junior']

client_df = client_df[~client_df['account_id'].isin(junior_clients['account_id'])]

transaction = transaction[~transaction['account_id'].isin(junior_clients['account_id'])]

client_df['has_card'] = client_df['card_id'].notnull()

print(f'Number of junior clients: {junior_clients.shape[0]}')
print(f'Number of clients remaining: {client_df.shape[0]}')

Number of junior clients: 145
Number of clients remaining: 4355


## Model Construction

### Processing Transactional Data
- The goal is to predict if a non-card-owner will buy a card or not

The first task is to look if every account in the transactions dataframe has a "first transaction". This would make the calculation of the monthly balance much easier since everything can be summed up without having to worry that there were months without records in the transaction dataframe.

In [18]:
# Find the minimum (first) transaction(s) date for each account
min_dates = transaction.groupby('account_id')['date'].min().reset_index()
min_dates.rename(columns={'date': 'min_date'}, inplace=True)

# Merge the minimum date back to the transactions to identify all transactions on the first day
transactions_with_min_date = pd.merge(transaction, min_dates, on='account_id')

# Filter transactions that are on the first day
first_day_transactions = transactions_with_min_date[transactions_with_min_date['date'] == transactions_with_min_date['min_date']]
first_day_transactions = first_day_transactions.copy()

# Now, for each of these first day transactions, check if any have amount equals balance
first_day_transactions['amount_equals_balance'] = first_day_transactions['amount'] == first_day_transactions['balance']

# Group by account_id and check if any transactions for each account meet the condition
accounts_meeting_condition = first_day_transactions.groupby('account_id')['amount_equals_balance'].any().reset_index()

# Verify if all accounts have at least one transaction on the first day meeting the condition
all_accounts_covered = accounts_meeting_condition['amount_equals_balance'].all()

print("Does every account's first day of transactions include at least one transaction where amount equals balance?", all_accounts_covered)

Does every account's first day of transactions include at least one transaction where amount equals balance? True


Now every accounts balance needs to be calculated per month.

In [19]:
transaction['month'] = transaction['date'].dt.to_period('M')

transactions_monthly = transaction.groupby(['account_id', 'month']).agg(
    volume=('amount', 'sum'),
    credit=('amount', lambda x: x[x > 0].sum()),
    withdrawal=('amount', lambda x: x[x < 0].sum()),
    n_transactions=('amount', 'size')
).reset_index()

In [20]:
transactions_monthly['month'] = pd.PeriodIndex(transactions_monthly['month'])

date_ranges = transactions_monthly.groupby('account_id')['month'].agg(['min', 'max'])

def reindex_df(group, account_id):
    idx = pd.period_range(start=group['month'].min(), end=group['month'].max(), freq='M')
    group.set_index('month', inplace=True)
    group = group.reindex(idx, fill_value=0)
    group.reset_index(inplace=True)
    group.rename(columns={'index': 'month'}, inplace=True)
    group['account_id'] = account_id
    return group

transactions_monthly = (transactions_monthly.groupby('account_id')
                        .apply(lambda x: reindex_df(x, x.name))
                        .reset_index(level=0, drop=True))

# Calculate cumulative balance
transactions_monthly['balance'] = transactions_monthly.groupby('account_id')['volume'].cumsum()

/var/folders/th/yt_529gn5qj62429rgx8mqzc0000gn/T/ipykernel_22690/3639110262.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: reindex_df(x, x.name))


In [21]:
transactions_monthly

,month,account_id,volume,credit,withdrawal,n_transactions,balance
0,1995-03,1,1000.0,1000.0,0.0,1,1000.0
1,1995-04,1,16298.2,16298.2,0.0,3,17298.2
2,1995-05,1,5858.0,5858.0,0.0,3,23156.2
3,1995-06,1,3979.6,3979.6,0.0,3,27135.8
4,1995-07,1,9087.9,9087.9,0.0,3,36223.7
...,...,...,...,...,...,...,...
36,1998-08,11382,54569.5,54569.5,0.0,7,2386853.6
37,1998-09,11382,44120.0,44120.0,0.0,5,2430973.6
38,1998-10,11382,63262.2,63262.2,0.0,6,2494235.8
39,1998-11,11382,50165.7,50165.7,0.0,5,2544401.5


### Defining Roll-Up Windows of Transactions

Before we can continue to filter out customers that have at least 13 Months of transaction history it is also needed to give non-customers a fictional `card_issued` date so we can build negative samples that also contain a 13-Month rollup window.

In [22]:
clients_with_cards = client_df[~client_df['card_issued'].isnull()]
time_between_creation_and_issue = (clients_with_cards['card_issued']-clients_with_cards['account_date']).dt.days

fig = go.Figure()

fig.add_trace(go.Histogram(x=time_between_creation_and_issue, histnorm='percent', nbinsx=50))

fig.update_traces(marker_line_width=1, marker_line_color="white")

fig.update_layout(
    title='Distribution of Card Issuance Dates',
    xaxis_title='Days',
    yaxis_title='Percentage of Accounts'
)

fig.show()

An observation we can make when looking at the distribution of when cards usually get issued after the account creation is that there don't seem to be any issuances before day 200 of the account.

In [23]:
np.random.seed(1337)

sampled_deltas = np.random.choice(time_between_creation_and_issue, size=len(client_df[client_df['card_issued'].isnull()]))

fig = go.Figure()

fig.add_trace(go.Histogram(x=sampled_deltas, histnorm='percent', name='Sampled Deltas', nbinsx=50))
fig.add_trace(go.Histogram(x=time_between_creation_and_issue, histnorm='percent', name='Original Deltas', nbinsx=50))

fig.update_traces(marker_line_width=1, marker_line_color="white")

fig.update_layout(
    title='Distribution of Card Issuance Dates',
    xaxis_title='Days',
    yaxis_title='Percentage of Accounts'
)

fig.show()

In [24]:
# fill the clients with missing card_issued dates with the sampled deltas
if len(client_df[client_df['card_issued'].isnull()]) > 0:
    client_df.loc[client_df['card_issued'].isnull(), 'card_issued'] = client_df.loc[client_df['card_issued'].isnull(), 'account_date'] + pd.to_timedelta(sampled_deltas, unit='D')
    
print(f'Number of NaT/NaN values in card_issued: {client_df["card_issued"].isnull().sum()}')

Number of NaT/NaN values in card_issued: 0


In [25]:
card_issued = card.groupby('disp_id')['card_issued'].min().reset_index()
card_issued.head(5)

,disp_id,card_issued
0,9,1998-10-16
1,19,1998-03-13
2,41,1995-09-03
3,42,1998-11-26
4,51,1995-04-24


In [26]:
client_df[client_df['disp_id'].isnull()].shape[0]

transactions_monthly.info()

<class 'pandas.core.frame.DataFrame'>
Index: 179054 entries, 0 to 40
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype    
---  ------          --------------   -----    
 0   month           179054 non-null  period[M]
 1   account_id      179054 non-null  int64    
 2   volume          179054 non-null  float64  
 3   credit          179054 non-null  float64  
 4   withdrawal      179054 non-null  float64  
 5   n_transactions  179054 non-null  int64    
 6   balance         179054 non-null  float64  
dtypes: float64(4), int64(2), period[M](1)
memory usage: 10.9 MB


In [27]:
# join in the card_issued date from client_date to the transactions_monthly
transactions_monthly = transactions_monthly.merge(client_df[['account_id', 'card_issued']], left_on='account_id', right_on='account_id', how='left')

transactions_monthly.head()

,month,account_id,volume,credit,withdrawal,n_transactions,balance,card_issued
0,1995-03,1,1000.0,1000.0,0.0,1,1000.0,1998-09-18
1,1995-04,1,16298.2,16298.2,0.0,3,17298.2,1998-09-18
2,1995-05,1,5858.0,5858.0,0.0,3,23156.2,1998-09-18
3,1995-06,1,3979.6,3979.6,0.0,3,27135.8,1998-09-18
4,1995-07,1,9087.9,9087.9,0.0,3,36223.7,1998-09-18


Now, let's see if the join worked correctly and we don't have any transactions without a `card_issued` date anymore.

In [28]:
assert transactions_monthly['card_issued'].isnull().sum() == 0

### Finding a way to impute card_issued date for non-buyers

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin

REFERENCE_DATE = client_df['card_issued'].dropna().mean()

class DateFeaturesExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, reference_date):
        self.reference_date = reference_date

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        for col in X.columns:
            X[col] = (pd.to_datetime(self.reference_date) - pd.to_datetime(X[col])).dt.days
        return X

datetime_cols = ['client_birth_date']
categorical_cols = ['client_gender', 'client_district_region', 'account_district_region']
numerical_cols = [col for col in client_df.columns if client_df[col].dtype in ['int64', 'float64']
                  and not col.endswith('_id')
                  and col not in ['ordertype_HOUSEHOLD', 'ordertype_INSURANCE PAYMENT', 'ordertype_LEASING', 'ordertype_LOAN PAYMENT', 'ordertype_MISSING']
                  and col not in datetime_cols]

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

date_transformer = DateFeaturesExtractor(reference_date=REFERENCE_DATE)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols),
        ('date', date_transformer, datetime_cols)
    ])

model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('model', KNeighborsRegressor(n_neighbors=10))
                       ])

X = client_df.drop(columns=['card_issued'])
y = client_df['card_issued'].fillna(pd.Timestamp(REFERENCE_DATE))

y = (pd.to_datetime(REFERENCE_DATE) - pd.to_datetime(y)).dt.days

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['client_age', 'loan_amount',
                                                   'loan_duration',
                                                   'loan_payments',
                                                   'district_id_x',
                                                   'client_district_population',
                                                   'client_district_n_municipalities_with_inhabitants_lt_499',
                                                   'client_district_n_municipalities_with_in...
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['client_gender',
                                                   'client_district_region',
                                                   'account_district_region']),
                                                 ('date',
                                                  DateFeaturesExtractor(reference_date=Timestamp('1997-10-01 14:14:24.661308800')),
                                                  ['client_birth_date'])])),
                ('model', KNeighborsRegressor(n_neighbors=10))])

In [30]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = model.predict(X_test)
predicted_dates = pd.to_datetime(REFERENCE_DATE) - pd.to_timedelta(y_pred, unit='d')
true_dates = pd.to_datetime(REFERENCE_DATE) - pd.to_timedelta(y_test, unit='d')
errors = (predicted_dates - true_dates).dt.days.abs()

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
average_error_days = errors.mean()

print(f'Mean Absolute Error (MAE): {mae:.2f} days')
print(f'Mean Squared Error (MSE): {mse:.2f} days^2')
print(f'Average error in days: {average_error_days:.2f} days')
print(f'Error distribution quantiles: {np.quantile(errors, [0.25, 0.5, 0.75])}')

Mean Absolute Error (MAE): 569.70 days
Mean Squared Error (MSE): 525794.88 days^2
Average error in days: 569.71 days
Error distribution quantiles: [198.5 469.  822. ]


In [31]:
transactions_monthly['card_issued'] = pd.to_datetime(transactions_monthly['card_issued'])
transactions_monthly['card_issued_period'] = transactions_monthly['card_issued'].dt.to_period('M')

transactions_monthly['month_diff'] = transactions_monthly.apply(lambda row: (row['card_issued_period'] - row['month']).n if pd.notnull(row['card_issued_period']) and pd.notnull(row['month']) else None, axis=1)

filtered_transactions = transactions_monthly[transactions_monthly['month_diff'].between(1, 13)]

In [32]:
transactions_monthly

,month,account_id,volume,credit,withdrawal,n_transactions,balance,card_issued,card_issued_period,month_diff
0,1995-03,1,1000.0,1000.0,0.0,1,1000.0,1998-09-18,1998-09,42
1,1995-04,1,16298.2,16298.2,0.0,3,17298.2,1998-09-18,1998-09,41
2,1995-05,1,5858.0,5858.0,0.0,3,23156.2,1998-09-18,1998-09,40
3,1995-06,1,3979.6,3979.6,0.0,3,27135.8,1998-09-18,1998-09,39
4,1995-07,1,9087.9,9087.9,0.0,3,36223.7,1998-09-18,1998-09,38
...,...,...,...,...,...,...,...,...,...,...
179049,1998-08,11382,54569.5,54569.5,0.0,7,2386853.6,1996-09-30,1996-09,-23
179050,1998-09,11382,44120.0,44120.0,0.0,5,2430973.6,1996-09-30,1996-09,-24
179051,1998-10,11382,63262.2,63262.2,0.0,6,2494235.8,1996-09-30,1996-09,-25
179052,1998-11,11382,50165.7,50165.7,0.0,5,2544401.5,1996-09-30,1996-09,-26


In [33]:
filtered_transactions.sort_values(by=['account_id', 'month_diff'])

,month,account_id,volume,credit,withdrawal,n_transactions,balance,card_issued,card_issued_period,month_diff
41,1998-08,1,6492.7,6492.7,0.0,5,338855.2,1998-09-18,1998-09,1
40,1998-07,1,6221.0,6221.0,0.0,4,332362.5,1998-09-18,1998-09,2
39,1998-06,1,6667.1,6667.1,0.0,5,326141.5,1998-09-18,1998-09,3
38,1998-05,1,6212.3,6212.3,0.0,4,319474.4,1998-09-18,1998-09,4
37,1998-04,1,7435.5,7435.5,0.0,6,313262.1,1998-09-18,1998-09,5
...,...,...,...,...,...,...,...,...,...,...
179017,1995-12,11382,82563.6,82563.6,0.0,4,221665.9,1996-09-30,1996-09,9
179016,1995-11,11382,51280.6,51280.6,0.0,4,139102.3,1996-09-30,1996-09,10
179015,1995-10,11382,49912.3,49912.3,0.0,4,87821.7,1996-09-30,1996-09,11
179014,1995-09,11382,37709.4,37709.4,0.0,4,37909.4,1996-09-30,1996-09,12


An issue at this point can be that an account may have months without any transactions, so there needs to be a more thorough process to interpolate the data:
- **Volume, Withdrawal, Credit and Number of Transactions**: If there are missing months in these variables we can just set `0` as their value as there has not been any activity if there were no recorded months.
- **Balance**: The balance will get recursively set to the last preceding recorded month. So if there are consecutive "missing" months in transactions the balance will always be set to the last recorded month.

In [34]:
account_summary = pd.DataFrame(filtered_transactions['account_id'].unique(), columns=['account_id'])

variables_to_pivot = ['volume', 'withdrawal', 'credit', 'n_transactions', 'balance']

for variable in variables_to_pivot:
    grouped = filtered_transactions.groupby(['account_id', 'month_diff'])[variable].sum().reset_index()
    pivot = grouped.pivot(index='account_id', columns='month_diff', values=variable).reset_index()
    pivot.columns = ['account_id'] + [f'{variable}_month_diff_{int(col)}' if col != 'account_id' else 'account_id' for col in pivot.columns[1:]]
    account_summary = pd.merge(account_summary, pivot, on='account_id', how='left')

for variable in ['volume', 'withdrawal', 'credit', 'n_transactions']:
    account_summary.update(account_summary.filter(regex=f'^{variable}_').fillna(0))

def find_last_balance(account_id, starting_month_diff):
    higher_month_diff = transactions_monthly[
        (transactions_monthly['account_id'] == account_id) & 
        (transactions_monthly['month_diff'] >= starting_month_diff)
    ].sort_values('month_diff')
    
    last_balance_row = higher_month_diff[higher_month_diff['balance'].notna()].head(1)
    
    if not last_balance_row.empty:
        return last_balance_row.iloc[0]['balance']
    else:
        return 0

balance_columns = [col for col in account_summary.columns if 'balance_month_diff_' in col]
for idx, row in account_summary.iterrows():
    for col in balance_columns:
        if pd.isna(row[col]):
            month_diff = int(col.split('_')[-1])
            last_balance = find_last_balance(row['account_id'], month_diff + 1)
            account_summary.at[idx, col] = last_balance

In [35]:
account_summary.head()

,account_id,volume_month_diff_1,volume_month_diff_2,volume_month_diff_3,volume_month_diff_4,volume_month_diff_5,volume_month_diff_6,volume_month_diff_7,volume_month_diff_8,volume_month_diff_9,volume_month_diff_10,volume_month_diff_11,volume_month_diff_12,volume_month_diff_13,withdrawal_month_diff_1,withdrawal_month_diff_2,withdrawal_month_diff_3,withdrawal_month_diff_4,withdrawal_month_diff_5,withdrawal_month_diff_6,withdrawal_month_diff_7,withdrawal_month_diff_8,withdrawal_month_diff_9,withdrawal_month_diff_10,withdrawal_month_diff_11,withdrawal_month_diff_12,withdrawal_month_diff_13,credit_month_diff_1,credit_month_diff_2,credit_month_diff_3,credit_month_diff_4,credit_month_diff_5,credit_month_diff_6,credit_month_diff_7,credit_month_diff_8,credit_month_diff_9,credit_month_diff_10,credit_month_diff_11,credit_month_diff_12,credit_month_diff_13,n_transactions_month_diff_1,n_transactions_month_diff_2,n_transactions_month_diff_3,n_transactions_month_diff_4,n_transactions_month_diff_5,n_transactions_month_diff_6,n_transactions_month_diff_7,n_transactions_month_diff_8,n_transactions_month_diff_9,n_transactions_month_diff_10,n_transactions_month_diff_11,n_transactions_month_diff_12,n_transactions_month_diff_13,balance_month_diff_1,balance_month_diff_2,balance_month_diff_3,balance_month_diff_4,balance_month_diff_5,balance_month_diff_6,balance_month_diff_7,balance_month_diff_8,balance_month_diff_9,balance_month_diff_10,balance_month_diff_11,balance_month_diff_12,balance_month_diff_13
0,1,6492.7,6221.0,6667.1,6212.3,7435.5,7018.6,6701.9,9091.5,10907.2,7318.0,6218.0,6600.6,8282.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6492.7,6221.0,6667.1,6212.3,7435.5,7018.6,6701.9,9091.5,10907.2,7318.0,6218.0,6600.6,8282.7,5.0,4.0,5.0,4.0,6.0,5.0,5.0,10.0,5.0,5.0,4.0,5.0,6.0,338855.2,332362.5,326141.5,319474.4,313262.1,305826.6,298808.0,292106.1,283014.6,272107.4,264789.4,258571.4,251970.8
1,2,34617.6,45943.4,52856.1,47098.6,52913.9,37980.7,31345.5,23949.5,1100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34617.6,45943.4,52856.1,47098.6,52913.9,37980.7,31345.5,23949.5,1100.0,0.0,0.0,0.0,0.0,6.0,7.0,7.0,6.0,3.0,3.0,3.0,3.0,1.0,0.0,0.0,0.0,0.0,327805.3,293187.7,247244.3,194388.2,147289.6,94375.7,56395.0,25049.5,1100.0,0.0,0.0,0.0,0.0
2,4,10917.6,12013.8,9011.7,14254.5,11343.0,9015.0,10769.2,9003.4,20249.4,13529.7,9040.6,9031.2,12329.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10917.6,12013.8,9011.7,14254.5,11343.0,9015.0,10769.2,9003.4,20249.4,13529.7,9040.6,9031.2,12329.5,6.0,6.0,5.0,6.0,7.0,5.0,6.0,5.0,12.0,7.0,5.0,5.0,6.0,195475.7,184558.1,172544.3,163532.6,149278.1,137935.1,128920.1,118150.9,109147.5,88898.1,75368.4,66327.8,57296.6
3,5,8607.1,7797.2,15720.3,10825.1,7812.5,5035.1,5017.0,5017.0,5017.0,5017.0,600.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8607.1,7797.2,15720.3,10825.1,7812.5,5035.1,5017.0,5017.0,5017.0,5017.0,600.0,0.0,0.0,5.0,4.0,10.0,6.0,4.0,3.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,76465.3,67858.2,60061.0,44340.7,33515.6,25703.1,20668.0,15651.0,10634.0,5617.0,600.0,0.0,0.0
4,6,14333.0,10810.0,10798.3,11989.9,20888.8,13055.1,10807.0,13039.2,10792.4,23857.9,11865.2,10815.6,10803.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14333.0,10810.0,10798.3,11989.9,20888.8,13055.1,10807.0,13039.2,10792.4,23857.9,11865.2,10815.6,10803.9,5.0,4.0,4.0,5.0,6.0,5.0,4.0,5.0,4.0,11.0,6.0,4.0,4.0,625481.9,611148.9,600338.9,589540.6,577550.7,556661.9,543606.8,532799.8,519760.6,508968.2,485110.3,473245.1,462429.5


In [36]:
client_df = client_df.merge(account_summary, on='account_id', how='inner')

### Mining Negative Examples
In order to find customers without a card to train our model with the following approach will look for for non-card-customers that show a *similar* activity pattern before the imputed `issue_date` to an existing card-owning-customer. An important note at this step is that it is crucial not to find an exact match of customers as that would introduce confusion in the following models. If we have exactly similar customers in our dataset the model will have a hard time to distinguish between card owners and non-card customers as there is no underlying pattern to learn.

In [37]:
account_summary_volume = account_summary.filter(regex='volume_month_diff_|account_id')

card_owners = client_df[client_df['has_card'] == True]
non_card_owners = client_df[client_df['has_card'] == False]

account_activity = pd.concat([account_summary_volume['account_id'], account_summary_volume.iloc[:, 1:] > 0], axis=1)
account_activity = account_activity.merge(client_df[['account_id', 'has_card']], on='account_id', how='left')

account_activity

,account_id,volume_month_diff_1,volume_month_diff_2,volume_month_diff_3,volume_month_diff_4,volume_month_diff_5,volume_month_diff_6,volume_month_diff_7,volume_month_diff_8,volume_month_diff_9,volume_month_diff_10,volume_month_diff_11,volume_month_diff_12,volume_month_diff_13,has_card
0,1,True,True,True,True,True,True,True,True,True,True,True,True,True,False
1,2,True,True,True,True,True,True,True,True,True,False,False,False,False,False
2,4,True,True,True,True,True,True,True,True,True,True,True,True,True,False
3,5,True,True,True,True,True,True,True,True,True,True,True,False,False,False
4,6,True,True,True,True,True,True,True,True,True,True,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3940,11333,True,True,True,True,True,True,True,True,True,True,True,True,True,False
3941,11349,True,True,True,True,True,True,True,True,True,True,True,False,False,False
3942,11359,True,True,True,True,True,True,True,True,False,False,False,False,False,True
3943,11362,True,True,True,True,True,True,True,True,True,True,True,True,True,False


In [38]:
from sklearn.metrics import pairwise_distances

def match_buyers_to_non_buyers_jaccard(buyers_df, non_buyers_df):
    # Convert boolean DataFrame to numpy arrays for compatibility with pairwise_distances
    buyer_activities = buyers_df.drop(columns=['account_id', 'has_card']).astype(bool).to_numpy()
    non_buyer_activities = non_buyers_df.drop(columns=['account_id', 'has_card']).astype(bool).to_numpy()

    # Compute the Jaccard distance matrix
    jaccard_dist_matrix = pairwise_distances(buyer_activities, non_buyer_activities, metric='jaccard')

    # Initialize lists to keep track of matched buyers and non-buyers
    matched_buyers_indices = []
    matched_non_buyers_indices = []

    # Sort buyers by the number of potential matches to prioritize those with fewer options
    potential_matches = np.argsort(-jaccard_dist_matrix.max(axis=1))

    for buyer_idx in potential_matches:
        # Find the non-buyer with the highest Jaccard similarity (lowest distance) that hasn't been matched yet
        potential_non_buyers = np.argsort(jaccard_dist_matrix[buyer_idx])
        for non_buyer_idx in potential_non_buyers:
            if non_buyer_idx not in matched_non_buyers_indices:
                # Found an unmatched non-buyer with the highest similarity for this buyer
                matched_buyers_indices.append(buyer_idx)
                matched_non_buyers_indices.append(non_buyer_idx)
                break

    matched_buyers_df = buyers_df.iloc[matched_buyers_indices].reset_index(drop=True)
    matched_non_buyers_df = non_buyers_df.iloc[matched_non_buyers_indices].reset_index(drop=True)

    return matched_buyers_df, matched_non_buyers_df

matched_buyers_df, matched_non_buyers_df = match_buyers_to_non_buyers_jaccard(account_activity[account_activity['has_card'] == True], account_activity[account_activity['has_card'] == False])

print(f'Number of matched buyers: {matched_buyers_df.shape[0]}')

Number of matched buyers: 747


In [43]:
matched_accounts = pd.concat([matched_buyers_df['account_id'], matched_non_buyers_df['account_id']])

client_df = client_df[client_df['account_id'].isin(matched_accounts)]

### Bringing the data together

In [45]:
n_lt_13_month_hist = transactions_monthly['account_id'].nunique()-filtered_transactions['account_id'].nunique()

preprocessing_summary = [
        n_merged_base_client,
        -len(junior_clients),
        -n_lt_13_month_hist,
        -len(non_card_owners)+len(matched_non_buyers_df),
        len(client_df)
]

fig = go.Figure(go.Waterfall(
        name = "20", orientation = "v",
        measure = ["absolute", "relative", "relative", "relative", "total"],
    x = ["Base Client List", "Junior Clients", "Clients with less than 13 Months of Transaction History", "Non-Card-Holders that were not matched on account activity", "Remaining Clients"],
    textposition = "outside",
    y = preprocessing_summary,
    text = [str(x) for x in preprocessing_summary],
    connector = {"line":{"color":"rgb(63, 63, 63)"}},
))

fig.update_layout(
        title = "Profit and loss statement 2018",
        showlegend = True
)

fig.show()

## Exploratory Data Analysis

### Exploring Missing Values

In [46]:
missing_values = client_df.isnull().sum()

missing_values = pd.DataFrame({
    'Number of Missing Values': missing_values[missing_values > 0],
    'Percentage of Missing Values': [
        f"{x:.2f}%" for x in (missing_values[missing_values > 0] / client_df.shape[0] * 100).round(2)
    ]
}, columns=['Number of Missing Values', 'Percentage of Missing Values'])

fig = go.Figure()

fig.add_trace(go.Bar(
    x=missing_values.index,
    y=missing_values['Number of Missing Values'],
    text=missing_values['Percentage of Missing Values'],
    textposition='auto'
))

fig.update_layout(
    title='Missing Values in the Dataset',
    xaxis_title='Columns',
    yaxis_title='Number of Missing Values'
)

fig.show()

### Cardholders vs Non-Cardholders

In [56]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=['Card Owners', 'Non-Card Owners'],
    y=[client_df[client_df['has_card'] == True].shape[0], client_df[client_df['has_card'] == False].shape[0]],
    text=[card_owners.shape[0], non_card_owners.shape[0]],
    textposition='auto'
))

fig.update_layout(
    title='Card Owners vs Non-Card Owners',
    xaxis_title='Card Ownership',
    yaxis_title='Number of Clients'
)

fig.show()

###  Card Types

In [57]:
card_types = card_owners['card_type'].value_counts()

fig = go.Figure()

fig.add_trace(go.Bar(
    x=card_types.index,
    y=card_types.values,
    text=card_types.values,
    textposition='auto'
))

fig.update_layout(
    title='Card Types of Card Owners',
    xaxis_title='Card Type',
    yaxis_title='Number of Clients'
)

fig.show()

### Volume before card purchase

In [49]:
fig = make_subplots(rows=2, cols=1, subplot_titles=('Volume of Transactions for Classic Card Holders', 'Volume of Transactions for Gold Card Holders'))

card_type_df = card_owners[card_owners['card_type'] == 'classic']
for col in [col for col in account_summary.columns if 'volume_month_diff_' in col]:
    month_num = col.split('_')[-1]
    fig.add_trace(go.Violin(
        x=[month_num]*card_type_df.shape[0],  # Use month number here
        y=card_type_df[col],
        name=month_num,  # Use month number as name
        box_visible=True,
        meanline_visible=True,
        showlegend=False
    ), row=1, col=1)

gold_card_type_df = card_owners[card_owners['card_type'] == 'gold']
for col in [col for col in account_summary.columns if 'volume_month_diff_' in col]:
    # Similarly, extract and use the month number
    month_num = col.split('_')[-1]
    fig.add_trace(go.Violin(
        x=[month_num]*gold_card_type_df.shape[0],
        y=gold_card_type_df[col],
        name=month_num,
        box_visible=True,
        meanline_visible=True,
        showlegend=False
    ), row=2, col=1)

fig.update_layout(
    title='Volume of Transactions by Card Type',
    xaxis_title='Months before purchase',
    xaxis2_title='Months before purchase',
    yaxis_title='Volume of Transactions',
    yaxis2_title='Volume of Transactions',
    height=1200
)

fig.show()

### Age distribution

In [50]:
fig = go.Figure()

fig.add_trace(go.Histogram(x=card_owners['client_age'], histnorm='percent', name='Card Owners', nbinsx=20))
fig.add_trace(go.Histogram(x=non_card_owners['client_age'], histnorm='percent', name='Non-Card Owners', nbinsx=20))

fig.update_traces(marker_line_width=1, marker_line_color="white")

fig.update_layout(
    title='Distribution of Client Age',
    xaxis_title='Age',
    yaxis_title='Percentage of Clients'
)

fig.show()

## Train-Test-Split

In [76]:
from sklearn.model_selection import train_test_split

X = client_df
y = X['has_card']
X = X.drop(columns=['has_card', 'card_id', 'card_issued', 'card_type'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)

In [77]:
print('Train set:')
print(y_train.value_counts(normalize=True))
print('\nTest set:')
print(y_test.value_counts(normalize=True))

Train set:
has_card
True     0.503766
False    0.496234
Name: proportion, dtype: float64

Test set:
has_card
False    0.51505
True     0.48495
Name: proportion, dtype: float64


## Baseline Model
Erstellen eines Baseline Modelles mittels logistischer Regression und den Informationen “Alter”,
“Geschlecht”, “Domizilregion”, “Vermögen” und “Umsatz” vor Kreditkartenkauf.

In [78]:
from sklearn.linear_model import LogisticRegression
from scripts.utils import create_pipeline

baseline_column_selection = client_df[['client_age', 'client_gender', 'client_district_region']]
client_vol_bal_cols = client_df.filter(regex='(volume|balance)', axis=1).columns

baseline_column_selection = pd.concat([baseline_column_selection, client_df[client_vol_bal_cols]], axis=1)

baseline_cat_columns = baseline_column_selection.select_dtypes(include=['object'])

base_pipeline = create_pipeline(baseline_cat_columns.columns, baseline_column_selection, LogisticRegression(max_iter=1000))

X_train, X_test = X_train[baseline_column_selection.columns], X_test[baseline_column_selection.columns]

In [79]:
base_pipeline = create_pipeline(categorical_features=baseline_cat_columns.columns.tolist(), 
                                numerical_features=baseline_column_selection.drop(baseline_cat_columns.columns, axis=1).columns.tolist(), 
                                estimator=LogisticRegression(max_iter=1000))

base_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cats',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['client_gender',
                                                   'client_district_region']),
                                                 ('nums',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['client_age',
                                                   'volume_month_diff_1',
                                                   'volume_month_diff_2',
                                                   'volume_month_diff_3',
                                                   'volume_month_diff...
                                                   'balance_month_diff_1',
                                                   'balance_month_diff_2',
                                                   'balance_month_diff_3',
                                                   'balance_month_diff_4',
                                                   'balance_month_diff_5',
                                                   'balance_month_diff_6',
                                                   'balance_month_diff_7',
                                                   'balance_month_diff_8',
                                                   'balance_month_diff_9',
                                                   'balance_month_diff_10',
                                                   'balance_month_diff_11',
                                                   'balance_month_diff_12',
                                                   'balance_month_diff_13'])])),
                ('model', LogisticRegression(max_iter=1000))])

In [80]:
from sklearn.metrics import classification_report

y_pred = base_pipeline.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.69      0.81      0.74       154
        True       0.75      0.61      0.67       145

    accuracy                           0.71       299
   macro avg       0.72      0.71      0.71       299
weighted avg       0.72      0.71      0.71       299



### Generating Event-Based Customer Information

## Feature Engineering

### Deriving New Features